In [1]:
import torch 
from torch import nn, optim
from torch.autograd import Variable
from tqdm import tqdm
from random import randint, shuffle
import string

use_cuda=True
# helper function to make one hot embedding when the alphabet is provided 
#alphabet is the set of uniq characters in your language
class OneHotEmbedding:
    def __init__(self, alphabet):
        self.alphabet = alphabet
        self.inverse_map = dict(enumerate(alphabet))
        self.map = dict(zip(alphabet, range(len(alphabet))))

    def N(self):
        return len(alphabet)

    def encode(self, x):
        # T x B x H = len(x) x 1 x N
        v = torch.FloatTensor(self.N()).zero_()
        v[self.map[x]] = 1
        return v
    
    def label(self, x):
        return torch.LongTensor([self.map[x]])

    def inverse_label(self, x):
        return self.inverse_map[x]

    def decode(self, y):
        _, max_probs = torch.max(y.transpose(0, 1), 2)
        max_probs = max_probs.squeeze()
        return self.inverse_classes(max_probs)
def generate(**kw):
    result = kw['prime']

    h = net.init_hidden()
    if use_cuda:
        h = h.cuda()

    x = None

    for char in result:
        x = onehot.encode(char)
        if use_cuda:
            x=x.cuda()
        x = Variable(x, requires_grad=False)
        y, h = net(x, h)

        
    for p in range(kw["length"]):
        y, h = net(x, h)
        y_dist = y.data.view(-1).div(kw["temperature"]).exp()
        argmax = torch.multinomial(y_dist, 1)[0]

        prediction = onehot.inverse_label(argmax)
        result += prediction
        x = onehot.encode(prediction)
        if use_cuda:
            x=x.cuda()
        x = Variable(x, requires_grad=False)
    return result


In [2]:
# Model Def
class Network(nn.Module):
    def __init__(self, **kw):
        super(Network, self).__init__()
        self.input_size = kw['input_size']
        self.hidden_size = kw['hidden_size']
        self.output_size = kw['output_size']
        self.n_layers = kw['n_layers']

        self.fc_in = nn.Linear(self.input_size, self.hidden_size)
        self.rnn = nn.GRU(self.hidden_size, self.hidden_size, self.n_layers)
        self.fc_out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, h):
        # One hot vector of single column coming in. 
        # View sorcery is to adjust to the layer's dimension requirement
        # Size(D) -> Size(1,D)

        x = self.fc_in(x.view(1, -1))

        # Mimicking TxBxD, required by RNN.
        # h(t-1) in, h(t) out.
        x, h = self.rnn(x.view(1, 1, -1), h)

        x = self.fc_out(x.view(1, -1))
        return x, h

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [3]:
net = torch.load("char_rnn_stateful_onehot_0_10000.pt")


In [4]:
use_cuda=True
printable=string.printable
#reads the text, make everything lower ( so that we will have lower class labels)
# and removes non printable characters from the corpus
text = open("../../../data/lab2/sh.txt").read().lower()

pruned_text = ''
for c in text:
    if c in printable and c not in '{}[]&_':
        pruned_text += c
text = pruned_text
alphabet = list(set(list(text)))

print ('size of your alphabet =', len(alphabet))
print ('your alphabet is =', alphabet)

onehot = OneHotEmbedding(alphabet)
new = generate(prime='elementary my dear watson'.lower(), temperature=0.1, length=100)

('size of your alphabet =', 52)
('your alphabet is =', ['\n', '!', ' ', '"', "'", ')', '(', '*', '-', ',', '/', '.', '1', '0', '3', '2', '5', '4', '7', '6', '9', '8', ';', ':', '?', 'a', '`', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z'])


RuntimeError: cuda runtime error (59) : device-side assert triggered at /OCRData/minesh.mathew/pytorch-0.1.12/torch/lib/THC/generic/THCStorage.c:32

In [8]:
new = generate(prime='elementary my dear watson'.lower(), temperature=0.1, length=100)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /OCRData/minesh.mathew/pytorch-0.1.12/torch/lib/THC/generic/THCTensorCopy.c:18

In [6]:
print (new)

elementary my dear watsoned to
     man which made and my looked and the man which had been to me to
     days and the mone a
